In [ ]:
import os
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import gradio as gr

# Azure OpenAI 설정
endpoint = os.getenv("ENDPOINT_URL", "https://team10-eighti.openai.azure.com/")
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o")
cognitiveServicesResource = os.getenv('AZURE_COGNITIVE_SERVICES_RESOURCE', 'YOUR_COGNITIVE_SERVICES_RESOURCE')

# Azure AD 토큰 제공자 생성
token_provider = get_bearer_token_provider(
    DefaultAzureCredential(),
    f'{cognitiveServicesResource}.default'
)

# Azure OpenAI 클라이언트 초기화
client = AzureOpenAI(
    azure_endpoint=endpoint,
    azure_ad_token_provider=token_provider,
    api_version='2024-05-01-preview',
)

# 채팅 함수 정의
def chat_with_openai(user_input, chat_history):
    try:
        # 이전 대화 기록 포함 메시지 구성
        messages = [{"role": "system", "content": "너는 수화를 알려주는 전문가야"}]
        for user_msg, assistant_msg in chat_history:
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": assistant_msg})
        messages.append({"role": "user", "content": user_input})

        # Azure OpenAI API 호출
        completion = client.chat.completions.create(
            model=deployment,
            messages=messages,
            max_tokens=800,
            temperature=0.7,
            top_p=0.95,
            frequency_penalty=0,
            presence_penalty=0,
            extra_body={
                "data_sources": [
                    {
                        "type": "azure_search",
                        "parameters": {
                            "endpoint": os.environ["AZURE_AI_SEARCH_ENDPOINT"],
                            "index_name": os.environ["AZURE_AI_SEARCH_INDEX"],
                            "authentication": {
                                "type": "azure_ad"
                            }
                        }
                    }
                ]
            }
        )

        # 응답 추출 및 대화 기록 업데이트
        assistant_reply = completion['choices'][0]['message']['content']
        chat_history.append((user_input, assistant_reply))
        return assistant_reply, chat_history

    except Exception as e:
        return f"Error: {str(e)}", chat_history

# Gradio 인터페이스 구성
with gr.Blocks() as demo:
    gr.Markdown("# Azure OpenAI Chatbot - 수화 전문가")
    chatbot = gr.Chatbot()
    user_input = gr.Textbox(label="Your Message", placeholder="메시지를 입력하세요...")
    clear_button = gr.Button("Clear Chat")
    
    # 대화 기록 저장
    state = gr.State([])

    # 이벤트 연결
    user_input.submit(chat_with_openai, [user_input, state], [chatbot, state])
    clear_button.click(lambda: None, None, chatbot)

# 실행
demo.launch()
